In [1]:

from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd

from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser_password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({})).iloc[:,1:] # Removes the object_id for each animal
#print(df)



#########################
# Dashboard Layout / View
#########################


app = JupyterDash('CS-340 - Project Two')
app.add_external_link = True


#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'grazioso_salvare_logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard -- Matt Oliphant'))),
    html.Center(html.Img(id='customer_image',
                         height='100',
                         width='100',
                         src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image')),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label':'Water Rescue', 'value':1},
                {'label':'Wilderness Rescue', 'value':2},
                {'label':'Disaster Rescue', 'value':3},
                {'label':'Reset', 'value':4}],
            #value=4,
            placeholder = 'Rescue Animals:',
            multi = False,
            clearable = False)
             
    ),
    html.Hr(),
    
    
    # this creates the datatable which will be updated when a dropdown item is selected
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
        #editable = False,
        #filter_action = 'native',
        #sort_action='native',
        #sort_mode = 'multi',
        #column_selectable='single',
        row_selectable='single',
        #row_deletable=False,
        #selected_columns=[],
        #selected_rows=[],
        #page_action='native',
        page_current = 0,
        page_size = 10,
    ),
    html.Br(),
    html.Hr(),
    
    
# this sets up the dashboard so that the pie chart and the geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 
        html.Div(
            dcc.Graph(id='graph-id'),
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


# this is by far the longest callback, but provides the queries for the dropdown options
# this callback will display the appropriate dogs that match the query according to the problem statement
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
### FIX ME Add code to filter interactive data table with MongoDB queries

    if (filter_type == 1):
        data = pd.DataFrame.from_records(
            shelter.read_all({'animal_type':'Dog',
                              'breed':{
                                  '$in':['Labrador Retriever Mix', 'Chesapeake Bay Retriever',
                                         'Newfoundland']},
                              'sex_upon_outcome':'Intact Female',
                              'age_upon_outcome_in_weeks':{'$gte':26}, 
                              'age_upon_outcome_in_weeks':{'$lte':156}}
                            )).iloc[:,1:].to_dict('records')
        
    if (filter_type == 2):
        data = pd.DataFrame.from_records(
            shelter.read_all({'animal_type':'Dog',
                              'breed':{
                                  '$in':['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog',
                                        'Siberian Husky', 'Rottweiler']},
                              'sex_upon_outcome':'Intact Male',
                              'age_upon_outcome_in_weeks':{'$gte':26},
                              'age_upon_outcome_in_weeks':{'$lte':156}}
                            )).iloc[:,1:].to_dict('records')
        
    if (filter_type == 3):
        data = pd.DataFrame.from_records(
            shelter.read_all({'animal_type':'Dog',
                              'breed':{
                                  '$in':['Doberman Pinscher', 'German Shepherd', 'Golden Retriever',
                                        'Bloodhound', 'Rottweiler']},
                              'sex_upon_outcome':'Intact Male',
                              'age_upon_outcome_in_weeks':{'$gte':20},
                              'age_upon_outcome_in_weeks':{'$lte':300}}
                            )).iloc[:,1:].to_dict('records')
        
    if (filter_type == 4):
        data = pd.DataFrame.from_records(shelter.read_all({})).iloc[:,1:].to_dict('records')
    
   
    
    columns=[{"name": i, "id": i} for i in df.columns]

    return (data, columns) # Return the chosen data query with its repective columns


# this callback highlight the row that is selected
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_style(selected_rows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]



# this callback will display a pie chart that displays breed stats of listed animals
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graph(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    return (
        px.pie(data_frame=dff, names='breed', color_discrete_sequence=px.colors.sequential.RdBu)
    )



# this callback will display a geolocation map of the first animal in the list
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data')])

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '600px', 'height': '400px'}, 
               center=[dff.loc[0, 'location_lat'], dff.loc[0, 'location_long']], zoom=9, children=[
            dl.TileLayer(id='base-layer-id'),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0, 'location_lat'], dff.loc[0, 'location_long']], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1('Animal Name'),
                    html.P(dff.iloc[0,10])                    
                ])
            ])
        ])
    ]


################
#  RUN THE APP
################

app
